# Importação de Bibliotecas

In [1]:
#  Importação de bibliotecas completas para experimentos com MLflow e DVC/Dagshub

# Manipulação de dados
import numpy as np
import pandas as pd

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# DVC / Dagshub para versionamento e conexão de dados
import dvc.api
from dagshub.data_engine import datasources
import dagshub

# Rastreamento de experimentos
import mlflow
import mlflow.sklearn
import mlflow.catboost
import mlflow.models.signature
from mlflow.models import infer_signature

# Machine Learning (modelos, treino, métricas)
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error,
    make_scorer
)

# Modelos básicos
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
import tempfile
import os
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
# Gradient Boosting Libraries
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from xgboost import XGBClassifier
# Utilidades do sistema
import os
import warnings
warnings.filterwarnings('ignore')
from tempfile import mkdtemp
print(" Todas as bibliotecas carregadas com sucesso!")


 Todas as bibliotecas carregadas com sucesso!


In [33]:
from mlflow.tracking import MlflowClient
import os
from dotenv import load_dotenv, find_dotenv 
from sklearn.model_selection import StratifiedKFold

In [2]:
from tempfile import mkdtemp

In [47]:
import sklearn.metrics as skm
from sklearn.metrics import make_scorer, recall_score

# Carregamento do Dataset Processado

Nesta etapa, vamos acessar o dataset processado diretamente do repositório versionado no **Dagshub/DVC**.  


In [3]:
# Carrega o dataset processado via Dagshub Data Engine
ds = datasources.get('estrellacouto05/quantum-finance-credit-score', 'processed')

# Exibe as primeiras linhas para verificação
ds.all().dataframe



Accessing as estrellacouto05

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,credit_score_processed.csv,103598542,https://dagshub.com/api/v1/repos/estrellacouto...,text/plain,32908761


Após obter o objeto `datasource`, agora vamos recuperar o link de download do dataset versionado.   
Com essa URL, carregamos os dados diretamente em um **DataFrame Pandas**.

In [4]:
# Obtém os primeiros registros do datasource e extrai o URL de download
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

# Carrega o dataset diretamente a partir do link de download
df = pd.read_csv(dataset_url)

# Exibe as primeiras linhas para verificação
df.head()


Output()

,idade,renda_anual,salario_liquido_mensal,qtd_contas_bancarias,qtd_cartoes_credito,taxa_juros,qtd_emprestimos,dias_atraso_pagamento,qtd_pagamentos_atrasados,variacao_limite_credito,...,tipos_emprestimos_Credit-Builder Loan,tipos_emprestimos_Debt Consolidation Loan,tipos_emprestimos_Home Equity Loan,tipos_emprestimos_Mortgage Loan,tipos_emprestimos_Not Specified,tipos_emprestimos_Payday Loan,tipos_emprestimos_Personal Loan,tipos_emprestimos_Student Loan,tipos_emprestimos_Two or More Types of Loan,score_credito
0,23.0,19114.12,1824.843333,3.0,4,3.0,4.0,3,7.0,11.27,...,False,False,False,False,False,False,False,False,True,2
1,23.0,19114.12,3335.886667,3.0,4,3.0,4.0,1,12.0,11.27,...,False,False,False,False,False,False,False,False,True,2
2,35.0,19114.12,3335.886667,3.0,4,3.0,4.0,3,7.0,9.40,...,False,False,False,False,False,False,False,False,True,2
3,23.0,19114.12,3335.886667,3.0,4,3.0,4.0,5,4.0,6.27,...,False,False,False,False,False,False,False,False,True,2
4,23.0,19114.12,1824.843333,3.0,4,3.0,4.0,6,12.0,11.27,...,False,False,False,False,False,False,False,False,True,2


# Desenvolvimento e experimentos de modelos

## Inicialização do Dagshub com MLflow

In [5]:
# Inicializa o Dagshub com integração ao MLflow
dagshub.init(repo_owner='estrellacouto05',
             repo_name='quantum-finance-credit-score',
             mlflow=True)

print("Dagshub inicializado e MLflow configurado.")


Initialized MLflow to track repo "estrellacouto05/quantum-finance-credit-score"

Repository estrellacouto05/quantum-finance-credit-score initialized!

Dagshub inicializado e MLflow configurado.


In [6]:
# Ativa o registro automático de experimentos com MLflow
mlflow.autolog()

print("MLflow Autolog habilitado.")


2025/08/27 14:40:35 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/08/27 14:40:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/08/27 14:40:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


MLflow Autolog habilitado.


## Separação de Features e Target



Nesta etapa, vamos separar as variáveis independentes (features) do target (`score_credito`).  
Isso organiza o dataset para os modelos de Machine Learning, que aprenderão a prever o target com base nas features.

In [7]:
# Lista todas as colunas e remove o target 'score_credito' da lista de features
features = list(df.columns)
features.remove("score_credito")

# Exibe o total de features selecionadas
print(f"Total de features: {len(features)}")


Total de features: 48


In [8]:
# Cria os DataFrames para features (X) e target (y)
X = df[features]
y = df["score_credito"]

# Exibe informações básicas para conferência
print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)


Dimensões de X: (100000, 48)
Dimensões de y: (100000,)


## Divisão dos Dados e Escalonamento

Nesta etapa, dividimos os dados em conjuntos de treino e teste para avaliar o desempenho dos modelos de forma justa.  
Utilizamos **70% para treino** e **30% para teste**.

Depois aplicamos o **RobustScaler**, que é ideal para dados com outliers, pois utiliza medianas e intervalos interquartis, evitando distorções.


In [9]:
# Divide os dados em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Conjuntos criados:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


Conjuntos criados:
X_train: (70000, 48), X_test: (30000, 48)
y_train: (70000,), y_test: (30000,)


In [10]:
# Inicializa o RobustScaler
scaler = RobustScaler()

# Ajusta o scaler apenas com os dados de treino e transforma
X_train_scaled = scaler.fit_transform(X_train)

# Aplica a mesma transformação nos dados de teste
X_test_scaled = scaler.transform(X_test)

print("Escalonamento concluído.")
print("Dimensões após escalonamento:", X_train_scaled.shape, X_test_scaled.shape)


Escalonamento concluído.
Dimensões após escalonamento: (70000, 48) (30000, 48)


In [24]:
X_train = X_train_scaled
X_test = X_test_scaled

## Função de Avaliação e Log no MLflow (Classificação)

- **Métricas principais:** precision, recall e f1-score para cada classe.
- **Destaque do recall da classe 0 (Poor)**, que é a mais importante.
- **Matriz de confusão** gerada como gráfico e registrada no MLflow como artefato.
- **Registro do modelo** no MLflow (CatBoost, XGBoost, LightGBM ou sklearn).

In [12]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    # Faz previsões
    predictions = model.predict(X_test)

    # Calcula métricas de classificação
    training_accuracy_score_manual = model.score(X_train, y_train)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions, labels=[0,1,2], zero_division=0)
    report = classification_report(y_test, predictions, digits=3)

    # Log de métricas macro (média das classes)
    mlflow.log_metric("Precision_macro", precision.mean())
    mlflow.log_metric("Recall_macro", recall.mean())
    mlflow.log_metric("F1_macro", f1.mean())

    # Log da acurácia de treino
    mlflow.log_metric("training_accuracy_score_manual", training_accuracy_score_manual)

    
    # Log detalhado de métricas por classe
    mlflow.log_metric("Recall_class_0_Poor", recall[0])
    mlflow.log_metric("Precision_class_0_Poor", precision[0])
    mlflow.log_metric("F1_class_0_Poor", f1[0])
    mlflow.log_metric("Recall_class_1_Standard", recall[1])
    mlflow.log_metric("Recall_class_2_Good", recall[2])

    # Cria assinatura do modelo para salvar no MLflow
    signature = infer_signature(X_test, predictions)

    # Salva o modelo de acordo com o tipo
    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    # Gera matriz de confusão
    cm = confusion_matrix(y_test, predictions, labels=[0,1,2])
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Poor', 'Standard', 'Good'], yticklabels=['Poor', 'Standard', 'Good'])
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.title(f'Matriz de Confusão - {model_name}')

    # Salva a figura temporariamente e envia como artefato para o MLflow
    with tempfile.TemporaryDirectory() as tmpdir:
        cm_path = os.path.join(tmpdir, "confusion_matrix.png")
        plt.savefig(cm_path)
        plt.close()
        mlflow.log_artifact(cm_path, artifact_path="confusion_matrix")

    # Print no console
    print(f"=== Avaliação do Modelo: {model_name} ===")
    print(report)
    print(f"Recall da classe 0 (Poor): {recall[0]:.3f}")
    print(f"Acurácia de Treino: {training_accuracy_score_manual:.3f}")
    

# Etapa de Treinamento de Modelos

A partir deste ponto, iniciamos o ciclo de **treinamento de modelos com MLflow**.

## XGBoost
  
Começamos com o **XGBoost Classifier**, um modelo de boosting altamente eficaz em dados tabulares.

- Foi usado **RandomizedSearchCV** com 30 combinações aleatórias de hiperparâmetros.
- Todas as métricas (precision, recall, f1-score) e a matriz de confusão foram registradas no MLflow.
- Nosso foco principal é **maximizar o recall da classe 0 (Poor)**, já que é a categoria mais crítica para o projeto.

Após avaliar o XGBoost, avançaremos para **LightGBM** e **CatBoost** para comparar os resultados.


In [26]:
with mlflow.start_run(run_name="XGBoost_Classifier_RandomSearch"):

    # Definindo a grade de hiperparâmetros ampliada
    param_distributions = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0],
        'min_child_weight': [1, 3, 5]
    }

    # Modelo base XGBoost
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

    # RandomizedSearchCV para explorar combinações de hiperparâmetros
    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_distributions,
        n_iter=30,                # número de combinações aleatórias a testar
        scoring='recall_macro',   # métrica principal
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treina com os dados escalonados
    random_search.fit(X_train_scaled, y_train)
    best_model = random_search.best_estimator_

    # Log dos melhores parâmetros no MLflow
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avalia e registra o modelo
    evaluate_and_log_model("xgboost", "XGBoost Classifier RandomSearch", best_model, X_test_scaled, y_test)

2025/08/03 22:02:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 30 candidates, totalling 150 fits


2025/08/03 22:13:25 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


=== Avaliação do Modelo: XGBoost Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.767     0.727     0.747      8805
           1      0.763     0.795     0.779     15873
           2      0.669     0.642     0.655      5322

    accuracy                          0.748     30000
   macro avg      0.733     0.721     0.727     30000
weighted avg      0.747     0.748     0.747     30000

Recall da classe 0 (Poor): 0.727
🏃 View run XGBoost_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/f9ec195a72dd4afb9b91f99ef5f727b2
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


## LightGBM 

Agora vamos treinar o **LightGBM Classifier**, outro modelo de boosting eficiente e mais rápido que o XGBoost em muitos cenários.

- Usaremos **RandomizedSearchCV** com 30 combinações, igual ao XGBoost, para manter o tempo de treino semelhante (~150 fits).
- Vamos incluir parâmetros específicos do LightGBM, como **num_leaves**, que controla a complexidade das árvores.
- Métrica de busca: **recall_macro**, priorizando melhor cobertura da classe “Poor”.
- Tudo será registrado no **MLflow** para comparação posterior.

In [28]:
with mlflow.start_run(run_name="LightGBM_Classifier_RandomSearch"):

    # Definição do espaço de busca
    param_distributions = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 5, 7, 9, -1],  # -1 = sem limite de profundidade
        'learning_rate': [0.01, 0.05, 0.1],
        'num_leaves': [15, 31, 63, 127],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0]
    }

    # Modelo base LightGBM
    lgb = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    # RandomizedSearchCV – 30 combinações x 5 folds = 150 fits
    random_search = RandomizedSearchCV(
        estimator=lgb,
        param_distributions=param_distributions,
        n_iter=30,
        scoring='recall_macro',
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treina o modelo
    random_search.fit(X_train_scaled, y_train)
    best_model = random_search.best_estimator_

    # Log dos melhores hiperparâmetros
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avalia e registra o modelo no MLflow
    evaluate_and_log_model("lightgbm", "LightGBM Classifier RandomSearch", best_model, X_test_scaled, y_test)


2025/08/03 22:21:43 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010766 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/03 22:30:14 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


=== Avaliação do Modelo: LightGBM Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.781     0.778     0.779      8805
           1      0.788     0.810     0.799     15873
           2      0.748     0.692     0.719      5322

    accuracy                          0.780     30000
   macro avg      0.773     0.760     0.766     30000
weighted avg      0.779     0.780     0.779     30000

Recall da classe 0 (Poor): 0.778
🏃 View run LightGBM_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/eaf41b22355a40d9a6a6b939818b364d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


Melhor foi light gbm 

##  Random Forest Classifier 

Neste experimento, aplicamos o modelo `RandomForestClassifier` para classificação dos scores de crédito, utilizando `RandomizedSearchCV` com validação cruzada (CV=5) para otimização de hiperparâmetros.

A escolha pelo `Random Forest` visa avaliar o desempenho de um modelo de ensemble tradicional, mais interpretável, como alternativa aos métodos baseados em Gradient Boosting (XGBoost e LightGBM).

**Configurações do experimento:**
- 50 combinações testadas com `RandomizedSearchCV` (`n_iter=50`)
- Métrica de avaliação principal: `recall_macro`
- Hiperparâmetros otimizados: número de estimadores, profundidade máxima (com limite para evitar overfitting), amostragem mínima para splits e folhas, tipo de critério, entre outros.
- Avaliação com função customizada `evaluate_and_log_model`, registrando métricas detalhadas (precision, recall, f1-score, matriz de confusão), com foco no `recall` da classe 0 (`Poor`), além de logging via MLflow.

O objetivo é verificar se o Random Forest entrega resultados competitivos e interpretar seu comportamento em comparação com os modelos anteriores.


In [ ]:
# Início do experimento com MLflow
with mlflow.start_run(run_name="RandomForest_Classifier_RandomSearch"):

    # Instancia o modelo base
    rf = RandomForestClassifier(random_state=42)

    # Espaço de busca ajustado (sem max_depth=None para evitar overfitting)
    param_distributions = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2'],
        'bootstrap': [True, False],
        'criterion': ['gini', 'entropy']
    }

    # RandomizedSearchCV com 50 combinações
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    # Treinamento
    random_search.fit(X_train_scaled, y_train)

    # Modelo final
    best_model = random_search.best_estimator_

    # Log dos melhores hiperparâmetros
    best_params = random_search.best_params_
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avaliação + log via função customizada
    evaluate_and_log_model("sklearn", "RandomForest Classifier RandomSearch", best_model, X_test_scaled, y_test)

2025/08/05 11:25:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 50 candidates, totalling 250 fits


2025/08/05 12:00:24 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


🏃 View run auspicious-deer-16 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/702b80c4c87b4456a148eb28e2e1b791
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run monumental-ox-450 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/83a8a9a3b726467fa8b2980745aa22ca
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run grandiose-zebra-368 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/8f247204dc2d4f8a9a0ff3d2547aa025
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run burly-ape-995 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/e8bf5ee20de9424099f1d17821de5018
🧪 View experiment at: https://dagsh

=== Avaliação do Modelo: RandomForest Classifier RandomSearch ===
              precision    recall  f1-score   support

           0      0.780     0.754     0.767      8805
           1      0.778     0.807     0.793     15873
           2      0.713     0.673     0.693      5322

    accuracy                          0.768     30000
   macro avg      0.757     0.745     0.751     30000
weighted avg      0.767     0.768     0.767     30000

Recall da classe 0 (Poor): 0.754
🏃 View run RandomForest_Classifier_RandomSearch at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/df711845800e452aa800d9578ce58457
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run enchanting-shad-321 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/1ab7344499f74817a85e876013514d9f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run serious-asp-250 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/c796d96f67054d22871e19825ce2175d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run invincible-rook-21 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/22ef267fe2ca4032a2ddcce5d44abc9a
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run angry-asp-955 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/7d9fc16a65f342b798a7e2299bdbfabd
🧪 View experiment at: https://dagshub

🏃 View run worried-steed-201 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/09b6d6b37e724f3a8a1da73fa71fc074
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run chill-mare-104 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/1ff41165c2554efe88c2927706ff5571
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run chill-smelt-845 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/e221b0dce6884322ad9a016113666026
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run big-asp-866 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/7bead2d39d444bf188e4735a0c8630a6
🧪 View experiment at: https://dagshub.com/est

## Fine-Tuning do LightGBM com RandomizedSearchCV

Nesta etapa, realizamos um ajuste fino (fine-tuning) no modelo LightGBM com foco em melhorar o **recall da classe 0 (Poor)**.  
Baseado nos melhores parâmetros anteriores, restringimos os ranges de busca para explorar variações próximas e mais promissoras.  
Usamos `RandomizedSearchCV` com `n_iter=50` e validação cruzada (`cv=5`), mantendo o controle de overfitting por meio de `num_leaves`, `subsample` e `colsample_bytree`.  
Os resultados serão registrados automaticamente no MLflow via integração com o Dagshub.


In [13]:
with mlflow.start_run(run_name="LightGBM_FineTuning"):

    model = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    param_distributions = {
        'n_estimators': [300, 400, 500],
        'learning_rate': [0.05, 0.08, 0.1],
        'num_leaves': [95, 127, 150],
        'max_depth': [5, 7, 9, -1],
        'subsample': [0.6, 0.7, 0.8],
        'colsample_bytree': [0.7, 0.85, 1.0]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        random_state=42,
        verbose=1,
        n_jobs=-1
    )

    randomized_search.fit(X_train_scaled, y_train)
    best_model = randomized_search.best_estimator_

    # Loga os melhores hiperparâmetros
    mlflow.log_params(randomized_search.best_params_)

    # Avalia e registra o modelo no MLflow
    evaluate_and_log_model("lightgbm", "LightGBM FineTuned", best_model, X_test_scaled, y_test)

2025/08/05 12:35:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/05 13:25:15 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


=== Avaliação do Modelo: LightGBM FineTuned ===
              precision    recall  f1-score   support

           0      0.785     0.786     0.785      8805
           1      0.793     0.812     0.803     15873
           2      0.755     0.701     0.727      5322

    accuracy                          0.784     30000
   macro avg      0.778     0.766     0.772     30000
weighted avg      0.784     0.784     0.784     30000

Recall da classe 0 (Poor): 0.786
🏃 View run LightGBM_FineTuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/05cfe5fa3fa544bbba01799cb7f7b903
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### Fine-tuning adicional no LightGBM com foco em `n_estimators` e `learning_rate`

Neste experimento, mantemos os melhores hiperparâmetros encontrados anteriormente (como `num_leaves`, `max_depth`, `subsample` e `colsample_bytree`) e realizamos um ajuste fino especificamente sobre os parâmetros `n_estimators` e `learning_rate`.

O objetivo é verificar se o aumento do número de estimadores (de 500 para 750 e 1000) combinado com taxas de aprendizado mais baixas (0.05, 0.03 e 0.01) resulta em ganho de recall, principalmente da classe 0 (Poor), sem causar overfitting.

A técnica continua sendo o `RandomizedSearchCV` com 50 iterações e validação cruzada (cv=5), mantendo o padrão adotado em experimentos anteriores.



In [14]:
with mlflow.start_run(run_name="LightGBM_Estimators_LearningRate_Tuning"):

    model = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

    param_distributions = {
        'n_estimators': [500, 750, 1000],
        'learning_rate': [0.05, 0.03, 0.01],
        'num_leaves': [127],
        'max_depth': [-1],
        'subsample': [0.7],
        'colsample_bytree': [1.0]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring='recall_macro',
        cv=5,
        random_state=42,
        verbose=1,
        n_jobs=-1
    )

    randomized_search.fit(X_train_scaled, y_train)
    best_model = randomized_search.best_estimator_

    # Log dos principais hiperparâmetros selecionados
    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)

    # Avaliação e log do modelo
    evaluate_and_log_model("lightgbm", "LightGBM Estimators-LR Tuning", best_model, X_test_scaled, y_test)

2025/08/05 13:57:07 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2708
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243159
[LightGBM] [Info] Start training from score -0.629475
[LightGBM] [Info] Start training from score -1.722287


2025/08/05 14:20:52 INFO mlflow.sklearn.utils: Logging the 5 best runs, 4 runs will be omitted.


=== Avaliação do Modelo: LightGBM Estimators-LR Tuning ===
              precision    recall  f1-score   support

           0      0.785     0.789     0.787      8805
           1      0.794     0.812     0.803     15873
           2      0.759     0.701     0.729      5322

    accuracy                          0.786     30000
   macro avg      0.779     0.767     0.773     30000
weighted avg      0.785     0.786     0.785     30000

Recall da classe 0 (Poor): 0.789
🏃 View run LightGBM_Estimators_LearningRate_Tuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/eb3c5dfe89f44412971d50969d0c882f
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


###  Escolha final do modelo LightGBM (Fine-Tuning)

Após dois ciclos de fine-tuning com o LightGBM, foram comparadas duas configurações principais:

| Modelo | `n_estimators` | `learning_rate` | Recall Classe 0 (Poor) |
|--------|----------------|------------------|--------------------------|
| A (1º Fine-tune) | **500** | **0.1** | **0.786** |
| B (2º Fine-tune) | **1000** | **0.05** | **0.789** |

Apesar do modelo B utilizar o dobro de árvores com um `learning_rate` mais baixo (0.05), **a melhora no recall da classe 0 foi mínima** (de 0.786 para 0.789), sem ganho significativo em outras métricas. Além disso, esse aumento nos estimadores **eleva o custo computacional e o risco de overfitting** sem retorno proporcional em desempenho.

####  Conclusão:
Optamos por manter o modelo com:
- `n_estimators = 500`
- `learning_rate = 0.1`

Essa combinação mostrou-se mais eficiente, com ótimo desempenho geral, menor complexidade e melhor equilíbrio entre performance e custo de treinamento.




## CatBoost

Agora vamos treinar o **CatBoost Classifier**, um modelo de boosting desenvolvido pela Yandex com excelente desempenho em bases com variáveis categóricas e estrutura mista (numéricas + booleanas).

- Usaremos o **RandomizedSearchCV** com 30 combinações, mantendo o padrão de experimentação adotado nos modelos anteriores.
- Serão explorados hiperparâmetros como **depth**, **learning_rate**, **l2_leaf_reg** e **border_count**.
- A métrica de otimização interna será **TotalF1**, adequada para tarefas multiclasse, enquanto a métrica de avaliação externa continua sendo o **recall_macro**.
- Toda a execução será rastreada e registrada no **MLflow**, com foco na classe “Poor” e comparação direta com os demais modelos.


In [18]:
catboost_temp_dir = mkdtemp()

with mlflow.start_run(run_name="CatBoost_FineTuning"):

    model = CatBoostClassifier(
        loss_function='MultiClass',
        eval_metric='TotalF1',  #  métrica escalar compatível
        verbose=0,
        train_dir=catboost_temp_dir,
        random_state=42
    )

    param_distributions = {
        'iterations': [300, 500, 750],
        'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7],
        'border_count': [32, 64, 128]
    }

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=30,
        scoring='recall_macro',  # usado para selecionar o melhor modelo
        cv=5,
        verbose=1,
        n_jobs=1,
        random_state=42
    )

    randomized_search.fit(X_train, y_train)

    best_model = randomized_search.best_estimator_

    # Log dos hiperparâmetros
    mlflow.log_param("best_iterations", best_model.get_params()["iterations"])
    mlflow.log_param("best_learning_rate", best_model.get_params()["learning_rate"])
    mlflow.log_param("best_depth", best_model.get_params()["depth"])
    mlflow.log_param("best_l2_leaf_reg", best_model.get_params()["l2_leaf_reg"])
    mlflow.log_param("best_border_count", best_model.get_params()["border_count"])

    # Avaliação
    evaluate_and_log_model("catboost", "CatBoost FineTuned", best_model, X_test, y_test)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


2025/08/05 18:17:53 INFO mlflow.sklearn.utils: Logging the 5 best runs, 25 runs will be omitted.


🏃 View run trusting-stoat-841 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/65d6e49d1b96461b845982489c694680
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run victorious-skunk-835 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/59239c09c853459c9889f22835e8195e
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run receptive-yak-708 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/ed4a8cce2fdf46fcb168768bb4ff358e
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run adaptable-finch-354 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/3738e2e53f624e8d9df1a46d6e113ddb
🧪 View experiment at: https:

=== Avaliação do Modelo: CatBoost FineTuned ===
              precision    recall  f1-score   support

           0      0.771     0.743     0.756      8805
           1      0.766     0.808     0.786     15873
           2      0.719     0.644     0.680      5322

    accuracy                          0.760     30000
   macro avg      0.752     0.732     0.741     30000
weighted avg      0.759     0.760     0.759     30000

Recall da classe 0 (Poor): 0.743
🏃 View run CatBoost_FineTuning at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/56d42725396f4f47b84222b54a4b5b1e
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


## Stacking Ensemble – LightGBM + CatBoost

Após avaliarmos isoladamente os modelos **LightGBM** e **CatBoost**, partiremos agora para a construção de um **modelo em ensemble via StackingClassifier**, com o objetivo de combinar suas forças.

- O **Stacking** é uma técnica de ensemble que combina diferentes algoritmos como base learners e utiliza um modelo meta para agregar suas previsões.
- Utilizaremos o **LightGBM** e o **CatBoost**, ambos já ajustados com **RandomizedSearchCV**, como base learners do ensemble.
- O meta-modelo escolhido será uma **árvore de decisão simples**, que aprenderá a combinar as predições das bases.
- Optamos por `passthrough=False` para usar apenas as predições dos modelos base como entrada para o meta-modelo, reduzindo risco de overfitting.
- O treinamento será feito sobre a mesma divisão `train/test`, e também avaliaremos a **robustez via validação cruzada externa** (5-fold).
- Toda a execução será rastreada pelo **MLflow**, incluindo métricas, matriz de confusão e parâmetros.

Essa abordagem visa obter um modelo mais generalizável e robusto, maximizando o desempenho sobre as diferentes classes.



In [19]:
with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta"):

    # Modelos base com hiperparâmetros ajustados
    estimators = [
        ("lgbm", LGBMClassifier(
            n_estimators=500,
            learning_rate=0.1,
            num_leaves=127,
            max_depth=-1,
            subsample=0.7,
            colsample_bytree=1.0,
            objective='multiclass',
            num_class=3,
            random_state=42
        )),
        ("catboost", CatBoostClassifier(
            iterations=750,
            learning_rate=0.05,
            depth=10,
            l2_leaf_reg=3,
            border_count=64,
            loss_function='MultiClass',
            eval_metric='TotalF1',
            verbose=0,
            random_state=42
        ))
    ]

    # Meta-modelo (estimador final)
    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    # Definição do ensemble com passthrough=False
    stacking_clf = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    # Treinamento
    stacking_clf.fit(X_train, y_train)

    # Avaliação com função central
    evaluate_and_log_model("stacking", "Stacking LGBM + CatBoost", stacking_clf, X_test, y_test)

    # Log extra
    mlflow.log_param("ensemble_method", "stacking")
    mlflow.log_param("meta_estimator", "random_forest")
    mlflow.log_param("passthrough", False)
    mlflow.log_param("cv", 5)


=== Avaliação do Modelo: Stacking LGBM + CatBoost ===
              precision    recall  f1-score   support

           0      0.768     0.842     0.804      8805
           1      0.833     0.773     0.802     15873
           2      0.727     0.768     0.747      5322

    accuracy                          0.792     30000
   macro avg      0.776     0.794     0.784     30000
weighted avg      0.795     0.792     0.793     30000

Recall da classe 0 (Poor): 0.842
🏃 View run Stacking_LGBM_CatBoost_RFMeta at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/c85566115777495aa40d6814d0bb7102
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### Justificativa para o Uso do Ensemble LightGBM + CatBoost

A decisão por utilizar um Stacking com **LightGBM** e **CatBoost** foi baseada nos resultados individuais obtidos após tuning cuidadoso:

- O **CatBoost FineTuned** obteve excelente performance, com **recall de 0.743 na classe 0 (Poor)** e uma média geral (`recall_macro`) de **0.732**, superando todos os modelos anteriores.
- O **LightGBM FineTuned** também apresentou bom desempenho, com **boa cobertura da classe 2 (Good)** e um tempo de treinamento significativamente inferior.
- Já o Stacking inicial com **três modelos (LightGBM, XGBoost e RandomForest)** teve desempenho muito abaixo do esperado, com recall da classe 0 em apenas 0.33 — sendo **descartado** por baixa eficácia e sobreposição de comportamento entre os modelos.
- Ao combinar apenas **os dois melhores modelos (CatBoost e LightGBM)**, alcançamos um **recall da classe 0 de 0.842** e um **recall_macro de 0.794**, além de uma acurácia de **0.792**.

Esse resultado evidencia que o ensemble entre esses dois algoritmos **potencializou os pontos fortes de cada um** e contribuiu para **uma classificação mais equilibrada entre as três classes**.

Com isso, este modelo passa a ser o **candidato principal à produção**, e será submetido à validação cruzada externa antes de ser registrado no **MLflow Model Registry**.


In [21]:
from sklearn.model_selection import cross_validate

# Métricas para avaliação externa
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Aplicando validação cruzada externa ao modelo já ajustado
cv_results = cross_validate(
    estimator=stacking_clf,
    X=X_train,
    y=y_train,
    cv=5,
    scoring=scoring,
    return_train_score=False,
    n_jobs=-1
)

# Exibindo resultados médios e variabilidade
print("🔎 Resultados da Validação Cruzada Externa (Stacking Final):\n")
for metric in scoring:
    media = cv_results[f'test_{metric}'].mean()
    desvio = cv_results[f'test_{metric}'].std()
    print(f"{metric:<18}: {media:.4f} ± {desvio:.4f}")


🔎 Resultados da Validação Cruzada Externa (Stacking Final):

accuracy          : 0.7810 ± 0.0033
precision_macro   : 0.7641 ± 0.0040
recall_macro      : 0.7801 ± 0.0047
f1_macro          : 0.7712 ± 0.0043


### Validação Cruzada Externa – Stacking LightGBM + CatBoost

Para garantir a robustez do ensemble construído com **LightGBM + CatBoost**, foi aplicada uma **validação cruzada externa (5-fold)** utilizando o `cross_validate` do `sklearn`.

- Essa abordagem avalia o modelo com diferentes divisões do dataset, oferecendo uma visão mais confiável sobre sua **generalização**.
- Foram calculadas as métricas: **accuracy**, **precision_macro**, **recall_macro** e **f1_macro**.
- Os resultados obtidos confirmam a **estabilidade e eficácia do ensemble**, com variações pequenas entre os folds.

###  Resultados da Validação Cruzada Externa (Stacking Final):

- **Accuracy**: `0.7810 ± 0.0033`  
- **Precision (Macro Avg)**: `0.7641 ± 0.0040`  
- **Recall (Macro Avg)**: `0.7801 ± 0.0047`  
- **F1-score (Macro Avg)**: `0.7712 ± 0.0043`

Esses resultados reforçam a escolha do modelo como **candidato final para produção**, com excelente equilíbrio entre as classes, especialmente em cenários multiclasse com forte desbalanceamento.


# Novo treinamento para melhora do modelo



## Análise Resumida dos Modelos no DagsHub
Esta cédula de código busca e compara os melhores modelos do seu experimento MLflow no DagsHub.

O script carrega suas credenciais de forma segura do arquivo .env.

Em seguida, classifica os modelos com base no recall da classe Poor, a métrica mais importante para o projeto.

Por fim, ele extrai e exibe uma tabela com as métricas-chave (como acurácia de treino e teste) e os principais parâmetros para te ajudar a identificar o melhor modelo e a causa do overfitting.

In [ ]:

# Encontra e carrega o arquivo .env, buscando a partir do diretório atual
load_dotenv(find_dotenv())

# Agora você pode acessar as variáveis de ambiente
mlflow_user = os.environ.get('MLFLOW_TRACKING_USERNAME')
mlflow_token = os.environ.get('MLFLOW_TRACKING_PASSWORD')

# A URI de rastreamento segue o padrão: https://dagshub.com/<usuario>/<repo>.mlflow
# O autolog já faz essa configuração para você, mas é bom ter o comando explícito aqui
mlflow.set_tracking_uri("https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow")
# 2. Inicializa o cliente do MLflow
client = MlflowClient()

# Define o nome do experimento
# Verifique o nome exato no seu Dashboard do DagsHub
experiment_name = "Default" # Geralmente é 'Default', a menos que você tenha criado um nome específico

try:
    experiment = client.get_experiment_by_name(experiment_name)
    if experiment is None:
        raise ValueError(f"Experiment with name '{experiment_name}' not found.")
    
    # Busca as 7 melhores execuções baseadas no recall da classe 0 (Poor)
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["metrics.Recall_class_0_Poor DESC"],
        max_results=7
    )

    model_data = []

    for run in runs:
        model_name = run.data.tags.get('mlflow.runName', run.info.run_name)

        metrics = run.data.metrics
        recall_poor = metrics.get('Recall_class_0_Poor', 'N/A')
        
        # O MLflow Autolog pode ter registrado a acurácia de treino e teste
        training_accuracy = metrics.get('training_accuracy_score', 'N/A')
        test_accuracy = metrics.get('accuracy', 'N/A')
        
        params = run.data.params
        
        relevant_params = {
            'learning_rate': params.get('learning_rate', 'N/A'),
            'n_estimators': params.get('n_estimators', 'N/A'),
            'num_leaves': params.get('num_leaves', 'N/A'),
            'depth': params.get('depth', 'N/A'),
            'l2_leaf_reg': params.get('l2_leaf_reg', 'N/A'),
            'meta_estimator': params.get('meta_estimator', 'N/A')
        }
        
        model_data.append({
            'Modelo': model_name,
            'Recall (Poor)': recall_poor,
            'Acurácia de Treino': training_accuracy,
            'Acurácia de Teste': test_accuracy,
            'Parâmetros': relevant_params
        })

    df = pd.DataFrame(model_data)

    print("Top 7 Modelos por Recall da Classe 'Poor' no DagsHub")
    print("-" * 60)
    print(df.to_markdown(index=False))

except ValueError as e:
    print(f"Erro: {e}")
except Exception as e:
    print(f"Ocorreu um erro ao conectar ou buscar no MLflow: {e}")



Top 7 Modelos por Recall da Classe 'Poor' no DagsHub
------------------------------------------------------------
| Modelo                                  |   Recall (Poor) |   Acurácia de Treino | Acurácia de Teste   | Parâmetros                                                                                                                                    |
|:----------------------------------------|----------------:|---------------------:|:--------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|
| Stacking_LGBM_CatBoost_RFMeta           |        0.842249 |             0.980586 | N/A                 | {'learning_rate': 'N/A', 'n_estimators': 'N/A', 'num_leaves': 'N/A', 'depth': 'N/A', 'l2_leaf_reg': 'N/A', 'meta_estimator': 'random_forest'} |
| LightGBM_Estimators_LearningRate_Tuning |        0.788984 |             0.996771 | N/A                 | {'learning_rate': 'N/A', '

## Stack com log mais paremtros 

In [ ]:
# Define os estimadores base
estimators = [
    ("lgbm", LGBMClassifier(random_state=42)),
    ("catboost", CatBoostClassifier(verbose=0, random_state=42))
]

# Define o meta-modelo
final_estimator = LogisticRegression(solver='lbfgs', multi_class='multinomial')

# Define o modelo de Stacking
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=5,
    n_jobs=-1
)

# Define a grade de parâmetros para a busca aleatória
param_dist = {
    'lgbm__n_estimators': np.arange(100, 1000, 100),
    'lgbm__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'lgbm__num_leaves': [16, 31, 63, 127],
    'catboost__iterations': np.arange(100, 1000, 100),
    'catboost__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'catboost__depth': [4, 6, 8, 10],
    'catboost__l2_leaf_reg': [1, 3, 5, 10],
    'final_estimator__C': [0.1, 1.0, 10.0]
}

# Configura o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=stacking_clf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='recall_macro',
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Inicia o run no MLflow e executa a busca
with mlflow.start_run(run_name="Stacking_RandomSearch_Pre_Scaled_Data"):
    random_search.fit(X_train_scaled, y_train)

    # Log dos melhores parâmetros e métricas no MLflow
    mlflow.log_params(random_search.best_params_)
    mlflow.log_metric("best_recall_macro_score", random_search.best_score_)

    # Exibe os melhores parâmetros e a melhor pontuação
    print("Melhores parâmetros encontrados:")
    print(random_search.best_params_)
    print(f"Melhor pontuação (recall macro): {random_search.best_score_:.4f}")
    
    # Armazena o melhor modelo treinado
    best_stacking_model = random_search.best_estimator_

2025/08/27 16:09:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 3 folds for each of 50 candidates, totalling 150 fits


2025/08/28 02:07:28 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


🏃 View run stately-wasp-233 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/f8142d7ae9b94f12aa778a340a89fed2
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run enchanting-worm-126 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/6941e0afad7b4e95ad3ca7e78b43a2ee
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run handsome-fish-556 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/5b0cb52c5b6c48f5b6a4fa1d4872a592
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run puzzled-wasp-608 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/d74a40c585fe440797909380818f5635
🧪 View experiment at: https://dags

🏃 View run bustling-hawk-256 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/cc878ab7d37c46fbaefa4689b06a1260
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


🏃 View run hilarious-bug-345 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/897ec702ca2d4e84bb48adc7c099f07a
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run wise-conch-344 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/9cbc6cbc4bc4426992d00bc41cc57196
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run bittersweet-cat-344 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/b34d53303ae0451281075ebee64f5d2d
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0
🏃 View run youthful-deer-665 at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/fcea06b95d1042758ed65b9b796d3a15
🧪 View experiment at: https://dagsh

### Fixação do resultado erro no log

In [27]:
# Início do experimento com MLflow para registrar o modelo final
with mlflow.start_run(run_name="Best_Stacking_Model_Final"):

    # Define os melhores parâmetros encontrados na busca
    best_params = {
        'lgbm__num_leaves': 63,
        'lgbm__n_estimators': 900,
        'lgbm__learning_rate': 0.2,
        'final_estimator__C': 0.1,
        'catboost__learning_rate': 0.01,
        'catboost__l2_leaf_reg': 5,
        'catboost__iterations': 100,
        'catboost__depth': 10
    }

    # Instancia os modelos base e o meta-modelo com os parâmetros otimizados
    lgbm_params = {k.replace('lgbm__', ''): v for k, v in best_params.items() if 'lgbm__' in k}
    catboost_params = {k.replace('catboost__', ''): v for k, v in best_params.items() if 'catboost__' in k}
    final_estimator_params = {k.replace('final_estimator__', ''): v for k, v in best_params.items() if 'final_estimator__' in k}

    lgbm = LGBMClassifier(**lgbm_params, random_state=42)
    catboost = CatBoostClassifier(**catboost_params, verbose=0, random_state=42)
    final_estimator = LogisticRegression(**final_estimator_params)
    
    # Define o modelo de Stacking final
    final_stacking_model = StackingClassifier(
        estimators=[("lgbm", lgbm), ("catboost", catboost)],
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1
    )

    # Treinamento do modelo final no conjunto de treino completo e escalonado
    final_stacking_model.fit(X_train_scaled, y_train)

    # Log dos melhores hiperparâmetros
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Avaliação e log via sua função customizada
    evaluate_and_log_model(
        "stacking", 
        "Modelo_Final_Producao", 
        final_stacking_model, 
        X_test_scaled, 
        y_test
    )

2025/08/28 11:08:52 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Modelo_Final_Producao ===
              precision    recall  f1-score   support

           0      0.791     0.764     0.778      8805
           1      0.778     0.824     0.800     15873
           2      0.769     0.676     0.720      5322

    accuracy                          0.780     30000
   macro avg      0.779     0.755     0.766     30000
weighted avg      0.780     0.780     0.779     30000

Recall da classe 0 (Poor): 0.764
🏃 View run Best_Stacking_Model_Final at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/55b2003a07654c12b99f81fe50641f8a
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


Não melhora com indice ruim 



## Stack em produção combate over

### primeira

In [ ]:
def _best_iters_lgbm(X, y, base_params, n_splits=5, patience=50, random_state=42):
    """Descobre melhores n_estimators (best_iteration_) para LGBM via CV com early stopping.
    Funciona com pandas DataFrame/Series OU NumPy arrays.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []

    for tr_idx, va_idx in skf.split(X, y):
        # Suporta pandas ou NumPy
        if hasattr(X, "iloc"):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        else:
            X_tr, X_va = X[tr_idx], X[va_idx]

        if hasattr(y, "iloc"):
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        else:
            y_tr, y_va = y[tr_idx], y[va_idx]

        lgbm = LGBMClassifier(**base_params)
        lgbm.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="multi_logloss",
            callbacks=[lgb.early_stopping(stopping_rounds=patience, verbose=False)]
        )

        it = getattr(lgbm, "best_iteration_", None)
        best_iters.append(int(it if it is not None else base_params.get("n_estimators", 500)))

    return int(np.median(best_iters))


def _best_iters_catboost(X, y, base_params, n_splits=5, patience=50, random_state=42):
    """Descobre melhores iterations (get_best_iteration) para CatBoost via CV com early stopping.
    Funciona com pandas DataFrame/Series OU NumPy arrays.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []

    for tr_idx, va_idx in skf.split(X, y):
        # Suporta pandas ou NumPy
        if hasattr(X, "iloc"):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        else:
            X_tr, X_va = X[tr_idx], X[va_idx]

        if hasattr(y, "iloc"):
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        else:
            y_tr, y_va = y[tr_idx], y[va_idx]

        cb = CatBoostClassifier(**base_params)
        cb.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            verbose=False,
            early_stopping_rounds=patience,
            use_best_model=True
        )

        it = cb.get_best_iteration()
        best_iters.append(int(it if it is not None and it > 0 else base_params.get("iterations", 750)))

    return int(np.median(best_iters))


with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned"):

    # ---------------------------
    # 1) Pré-ajuste com early stopping (não mexe no Stacking ainda)
    # ---------------------------
    lgbm_base_params = dict(
        n_estimators=500,
        learning_rate=0.1,
        num_leaves=127,
        max_depth=-1,
        subsample=0.7,
        colsample_bytree=1.0,
        objective='multiclass',
        num_class=3,
        random_state=42
    )

    cat_base_params = dict(
        iterations=750,
        learning_rate=0.05,
        depth=10,
        l2_leaf_reg=3,
        border_count=64,
        loss_function='MultiClass',
        eval_metric='TotalF1',
        verbose=0,
        random_state=42
    )

    # Descobrir melhores iterações por CV com early stopping
    best_lgbm_iters = _best_iters_lgbm(
        X_train, y_train,
        base_params=lgbm_base_params,
        n_splits=5,
        patience=50,
        random_state=42
    )
    best_cat_iters = _best_iters_catboost(
        X_train, y_train,
        base_params=cat_base_params,
        n_splits=5,
        patience=50,
        random_state=42
    )

    # Log dos melhores valores encontrados
    mlflow.log_param("early_stop_search_cv", 5)
    mlflow.log_param("early_stop_patience", 50)
    mlflow.log_param("best_lgbm_n_estimators", best_lgbm_iters)
    mlflow.log_param("best_catboost_iterations", best_cat_iters)

    # ---------------------------
    # 2) Recriar base learners com iterações reduzidas
    # ---------------------------
    lgbm_final = LGBMClassifier(
        **{**lgbm_base_params, "n_estimators": best_lgbm_iters}
    )

    catboost_final = CatBoostClassifier(
        **{**cat_base_params, "iterations": best_cat_iters}
    )

    estimators = [
        ("lgbm", lgbm_final),
        ("catboost", catboost_final)
    ]

    # ---------------------------
    # 3) Meta-modelo e Stacking
    # ---------------------------
    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    stacking_clf = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    # Treinamento final do ensemble
    stacking_clf.fit(X_train, y_train)

    # ---------------------------
    # 4) Avaliação + MLflow
    # ---------------------------
    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM+CatBoost (early-stop tuned) → RF meta",
        stacking_clf,
        X_test, y_test
    )

    # Logs extras
    mlflow.log_param("ensemble_method", "stacking")
    mlflow.log_param("meta_estimator", "random_forest")
    mlflow.log_param("passthrough", False)
    mlflow.log_param("cv", 5)
    mlflow.log_param("lgbm_learning_rate", lgbm_base_params["learning_rate"])
    mlflow.log_param("lgbm_num_leaves", lgbm_base_params["num_leaves"])
    mlflow.log_param("catboost_depth", cat_base_params["depth"])
    mlflow.log_param("catboost_l2_leaf_reg", cat_base_params["l2_leaf_reg"])


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005810 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2704
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243184
[LightGBM] [Info] Start training from score -0.629468
[LightGBM] [Info] Start training from score -1.722267
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008432 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2707
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243184
[LightGBM] [Info] Start training from score -0.629468
[LightGBM] [Info] 

2025/08/28 12:08:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Stacking LGBM+CatBoost (early-stop tuned) → RF meta ===
              precision    recall  f1-score   support

           0      0.770     0.814     0.791      8805
           1      0.815     0.780     0.797     15873
           2      0.722     0.748     0.735      5322

    accuracy                          0.784     30000
   macro avg      0.769     0.780     0.774     30000
weighted avg      0.785     0.784     0.784     30000

Recall da classe 0 (Poor): 0.814
🏃 View run Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/a2bf38a68ba3447882634914df519bf9
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### Segunda


### A

In [38]:
def _best_iters_lgbm(X, y, base_params, n_splits=5, patience=40, random_state=42):
    """Estimativa do melhor n_estimators para LGBM via CV com early stopping.
    Funciona com pandas ou NumPy.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        if hasattr(X, "iloc"):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        else:
            X_tr, X_va = X[tr_idx], X[va_idx]

        if hasattr(y, "iloc"):
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        else:
            y_tr, y_va = y[tr_idx], y[va_idx]

        mdl = LGBMClassifier(**base_params)
        mdl.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="multi_logloss",
            callbacks=[lgb.early_stopping(stopping_rounds=patience, verbose=False)]
        )
        it = getattr(mdl, "best_iteration_", None)
        best_iters.append(int(it if it is not None else base_params.get("n_estimators", 500)))
    return int(np.median(best_iters))


with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_LGBMReg_2A"):
    # ---------------------------
    # 1) LGBM com regularização leve (apertos sugeridos)
    # ---------------------------
    lgbm_reg_params = dict(
        # manter learning_rate e subsample base
        learning_rate=0.1,
        subsample=0.7,
        subsample_freq=1,         # ativa bagging
        colsample_bytree=0.8,     # feature_fraction
        # capacidade/complexidade
        num_leaves=95,            # ↓ de 127
        max_depth=10,             # antes -1
        min_child_samples=60,     # ↑ de 20
        # regularização explícita
        reg_lambda=1.0,           # L2
        reg_alpha=0.1,            # L1
        min_split_gain=0.1,
        # demais
        objective="multiclass",
        num_class=3,
        random_state=42,
        n_estimators=600          # valor provisório; será recalibrado via early stopping
    )

    # 2) Re-otimizar n_estimators do LGBM com ES (CV=5, patience=40)
    best_lgbm_iters_2a = _best_iters_lgbm(
        X_train, y_train,
        base_params=lgbm_reg_params,
        n_splits=5,
        patience=40,
        random_state=42
    )

    mlflow.log_param("stage", "2A_LGBM_regularized")
    mlflow.log_param("early_stop_search_cv", 5)
    mlflow.log_param("early_stop_patience", 40)
    mlflow.log_param("best_lgbm_n_estimators_2A", best_lgbm_iters_2a)

    # ---------------------------
    # 3) CatBoost reaproveitado (fallback para 750 se não foi definido antes)
    # ---------------------------
    try:
        best_cat_iters  # se você definiu antes (ex.: best_cat_iters = 6XX)
    except NameError:
        best_cat_iters = 750  # fallback seguro

    cat_params = dict(
        iterations=best_cat_iters,
        learning_rate=0.05,
        depth=10,
        l2_leaf_reg=3,
        border_count=64,
        loss_function="MultiClass",
        eval_metric="TotalF1",
        verbose=0,
        random_state=42
    )
    mlflow.log_param("catboost_iterations_used", best_cat_iters)

    # ---------------------------
    # 4) Montar ensemble e treinar
    # ---------------------------
    lgbm_final_2a = LGBMClassifier(**{**lgbm_reg_params, "n_estimators": best_lgbm_iters_2a})
    catboost_final = CatBoostClassifier(**cat_params)

    estimators = [
        ("lgbm", lgbm_final_2a),
        ("catboost", catboost_final)
    ]

    final_estimator = RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        random_state=42
    )

    stacking_clf_2a = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )

    stacking_clf_2a.fit(X_train, y_train)

    # ---------------------------
    # 5) Avaliação + MLflow
    # ---------------------------
    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM+CatBoost (LGBM regularizado + ES retuned) → RF meta",
        stacking_clf_2a,
        X_test, y_test
    )

    # Log dos hiperparâmetros chave do LGBM (para rastreio fácil)
    mlflow.log_param("lgbm_num_leaves_2A", lgbm_reg_params["num_leaves"])
    mlflow.log_param("lgbm_max_depth_2A", lgbm_reg_params["max_depth"])
    mlflow.log_param("lgbm_min_child_samples_2A", lgbm_reg_params["min_child_samples"])
    mlflow.log_param("lgbm_reg_lambda_2A", lgbm_reg_params["reg_lambda"])
    mlflow.log_param("lgbm_reg_alpha_2A", lgbm_reg_params["reg_alpha"])
    mlflow.log_param("lgbm_min_split_gain_2A", lgbm_reg_params["min_split_gain"])
    mlflow.log_param("lgbm_colsample_bytree_2A", lgbm_reg_params["colsample_bytree"])
    mlflow.log_param("lgbm_subsample_2A", lgbm_reg_params["subsample"])
    mlflow.log_param("lgbm_subsample_freq_2A", lgbm_reg_params["subsample_freq"])


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015822 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2704
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 48
[LightGBM] [Info] Start training from score -1.243184
[LightGBM] [Info] Start training from score -0.629468
[LightGBM] [Info] Start training from score -1.722267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

2025/08/28 17:37:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


=== Avaliação do Modelo: Stacking LGBM+CatBoost (LGBM regularizado + ES retuned) → RF meta ===
              precision    recall  f1-score   support

           0      0.765     0.818     0.790      8805
           1      0.819     0.777     0.798     15873
           2      0.727     0.754     0.740      5322

    accuracy                          0.785     30000
   macro avg      0.770     0.783     0.776     30000
weighted avg      0.787     0.785     0.785     30000

Recall da classe 0 (Poor): 0.818
Acurácia de Treino: 0.919
🏃 View run Stacking_LGBM_CatBoost_RFMeta_LGBMReg_2A at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/77df8329df6c46238fec76af0aeb3fa3
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


### B

In [48]:
# CatBoost: iterações fixadas a partir do MLflow (run "Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned" -> param "best_catboost_iterations" = 747)
best_cat_iters = 747
assert isinstance(best_cat_iters, int) and best_cat_iters > 0

def _best_iters_lgbm(X, y, base_params, n_splits=5, patience=35, random_state=42):
    """Reestima o melhor n_estimators via CV com early stopping (suporta pandas/NumPy)."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_iters = []
    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = (X.iloc[tr_idx], X.iloc[va_idx]) if hasattr(X, "iloc") else (X[tr_idx], X[va_idx])
        y_tr, y_va = (y.iloc[tr_idx], y.iloc[va_idx]) if hasattr(y, "iloc") else (y[tr_idx], y[va_idx])

        mdl = LGBMClassifier(**base_params)
        mdl.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="multi_logloss",
            callbacks=[lgb.early_stopping(stopping_rounds=patience, verbose=False)]
        )
        it = getattr(mdl, "best_iteration_", None)
        best_iters.append(int(it if it is not None else base_params.get("n_estimators", 350)))
    return int(np.median(best_iters))

def _recall_poor(y_true, y_pred):
    """Recall da classe 0 (Poor) para monitoramento secundário na busca."""
    per_class = recall_score(y_true, y_pred, labels=[0, 1, 2], average=None, zero_division=0)
    return float(per_class[0])

recall_macro_scorer = make_scorer(recall_score, average="macro", zero_division=0)
recall_poor_scorer  = make_scorer(_recall_poor)

with mlflow.start_run(run_name="Stacking_LGBM_CatBoost_RFMeta_LGBMRandomSearch_2B_PlanA"):
    # Metadados e rastreabilidade
    mlflow.log_param("stage", "2B_LGBM_random_search_PlanA")
    mlflow.log_param("budget_n_iter", 12)
    mlflow.log_param("budget_cv_search", 3)
    mlflow.log_param("budget_cv_es", 5)
    mlflow.set_tag("catboost_iters_source", "MLflow: Stacking_LGBM_CatBoost_RFMeta_EarlyStopTuned::best_catboost_iterations=747")
    mlflow.log_param("catboost_iterations_used", best_cat_iters)

    # Base do LGBM na busca (sem early stopping; n_estimators moderado para acelerar)
    lgbm_base_fixed = dict(
        learning_rate=0.1,
        objective="multiclass",
        num_class=3,
        random_state=42,
        n_estimators=350,   # menor que 500 para reduzir custo por fit; será reotimizado depois
        verbosity=-1
    )

    # Espaço de busca focado ao redor do 2.A
    param_distributions = {
        "num_leaves":        [80, 85, 90, 95, 100, 105, 110],
        "max_depth":         [8, 9, 10, 11, 12],
        "min_child_samples": [40, 50, 60, 70, 80],
        "reg_lambda":        [0.5, 0.75, 1.0, 1.5, 2.0, 3.0],
        "reg_alpha":         [0.05, 0.1, 0.2, 0.3, 0.5],
        "min_split_gain":    [0.0, 0.05, 0.1, 0.15],
        "colsample_bytree":  [0.75, 0.8, 0.85, 0.9],
        "subsample":         [0.6, 0.65, 0.7, 0.75, 0.8],
        "subsample_freq":    [1, 2, 3],
    }

    # Busca enxuta: 12 amostras x CV=3 = 36 fits
    lgbm_search_model = LGBMClassifier(**lgbm_base_fixed)
    skf_search = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    search = RandomizedSearchCV(
        estimator=lgbm_search_model,
        param_distributions=param_distributions,
        n_iter=12,
        scoring={"recall_macro": recall_macro_scorer, "recall_poor": recall_poor_scorer},
        refit=False,               # evita 1 fit extra
        cv=skf_search,
        n_jobs=-1,
        verbose=1,
        random_state=42,
        return_train_score=False
    )
    search.fit(X_train, y_train)

    # Seleção manual do melhor por recall_macro e logging dos resultados
    results_df = pd.DataFrame(search.cv_results_).copy()
    results_df.sort_values("mean_test_recall_macro", ascending=False, inplace=True)
    best_idx = int(np.argmax(search.cv_results_["mean_test_recall_macro"]))
    best_recall_macro = float(search.cv_results_["mean_test_recall_macro"][best_idx])
    best_recall_poor  = float(search.cv_results_["mean_test_recall_poor"][best_idx])
    mlflow.log_metric("best_cv_recall_macro_2B", best_recall_macro)
    mlflow.log_metric("best_cv_recall_poor_2B", best_recall_poor)

    # Log dos melhores hiperparâmetros da busca
    best_params_search = results_df.iloc[0]["params"]
    for k, v in best_params_search.items():
        mlflow.log_param(f"lgbm_best_{k}_2B", v)

    # Artefatos: resultados completos e top-5
    with tempfile.TemporaryDirectory() as tmpdir:
        all_path  = os.path.join(tmpdir, "lgbm_random_search_results_full.csv")
        top5_path = os.path.join(tmpdir, "lgbm_random_search_top5.csv")
        results_df.to_csv(all_path, index=False)
        results_df.head(5).to_csv(top5_path, index=False)
        mlflow.log_artifact(all_path,  artifact_path="random_search")
        mlflow.log_artifact(top5_path, artifact_path="random_search")

    # Retune de n_estimators do LGBM com early stopping (CV=5; ~5 fits)
    best_params = {**lgbm_base_fixed, **best_params_search, "n_estimators": 500}  # provisório para ES
    best_lgbm_iters_2B = _best_iters_lgbm(
        X_train, y_train,
        base_params=best_params,
        n_splits=5,
        patience=35,
        random_state=42
    )
    mlflow.log_param("best_lgbm_n_estimators_2B", int(best_lgbm_iters_2B))

    # Ensemble final: LGBM vencedor + CatBoost fixo + RF meta (Stacking CV=5; ~13 fits)
    lgbm_final_2b = LGBMClassifier(**{**best_params, "n_estimators": int(best_lgbm_iters_2B)})
    catboost_final = CatBoostClassifier(
        iterations=best_cat_iters,
        learning_rate=0.05,
        depth=10,
        l2_leaf_reg=3,
        border_count=64,
        loss_function="MultiClass",
        eval_metric="TotalF1",
        verbose=0,
        random_state=42
    )
    estimators = [("lgbm", lgbm_final_2b), ("catboost", catboost_final)]
    final_estimator = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)

    stacking_clf_2b = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1,
        passthrough=False
    )
    stacking_clf_2b.fit(X_train, y_train)

    # Avaliação e logging no MLflow
    evaluate_and_log_model(
        "stacking",
        "Stacking LGBM+CatBoost (LGBM RandomSearch PlanA + ES retuned) → RF meta",
        stacking_clf_2b,
        X_test, y_test
    )

    # Logging de parâmetros-chave do LGBM (para rastreabilidade)
    for hp in ["num_leaves", "max_depth", "min_child_samples", "reg_lambda", "reg_alpha",
               "min_split_gain", "colsample_bytree", "subsample", "subsample_freq"]:
        mlflow.log_param(f"lgbm_{hp}_2B_final", getattr(lgbm_final_2b, hp))

2025/08/28 18:51:42 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fitting 3 folds for each of 12 candidates, totalling 36 fits


2025/08/28 18:58:51 WARNING mlflow.sklearn.utils: Failed to autolog metrics for RandomizedSearchCV. Logging error: This 'RandomizedSearchCV' has no attribute 'predict'
2025/08/28 18:58:51 WARNING mlflow.sklearn.utils: _get_classifier_artifacts.<locals>.plot_confusion_matrix failed. The artifact training_confusion_matrix will not be recorded. Artifact error: This 'RandomizedSearchCV' has no attribute 'predict'
2025/08/28 18:59:10 WARNING mlflow.sklearn.utils: BaseSearchCV.score failed. The 'training_score' metric will not be recorded. Scoring error: This RandomizedSearchCV instance was initialized with `refit=False`. score is available only after refitting on the best parameters. You can refit an estimator manually using the `best_params_` attribute
2025/08/28 18:59:11 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2025/08/28 18:59:11 WARNING mlflow.sklearn: M

=== Avaliação do Modelo: Stacking LGBM+CatBoost (LGBM RandomSearch PlanA + ES retuned) → RF meta ===
              precision    recall  f1-score   support

           0      0.768     0.821     0.793      8805
           1      0.821     0.779     0.799     15873
           2      0.730     0.759     0.744      5322

    accuracy                          0.788     30000
   macro avg      0.773     0.786     0.779     30000
weighted avg      0.789     0.788     0.788     30000

Recall da classe 0 (Poor): 0.821
Acurácia de Treino: 0.938
🏃 View run Stacking_LGBM_CatBoost_RFMeta_LGBMRandomSearch_2B_PlanA at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0/runs/74733e4ed7c94616ad33400c8c834bc5
🧪 View experiment at: https://dagshub.com/estrellacouto05/quantum-finance-credit-score.mlflow/#/experiments/0


# Registro de Modelo em Produção

Após treinar e avaliar o modelo, podemos registrá-lo oficialmente no **Model Registry do MLflow**.  
Isso permite versionar o modelo, promovê-lo para produção e gerenciar futuras atualizações.  

- Usamos o **run_id** obtido no link do MLflow (na interface Dagshub).
- Escolhemos um nome amigável e consistente para o modelo, neste caso: `credit-score-model`.

In [22]:
run_id = "c85566115777495aa40d6814d0bb7102"

mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name="credit-score-model"
)

print(f" Modelo registrado como 'credit-score-model' (run_id: {run_id})")

Registered model 'credit-score-model' already exists. Creating a new version of this model...
2025/08/05 20:19:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: credit-score-model, version 2
Created version '2' of model 'credit-score-model'.


 Modelo registrado como 'credit-score-model' (run_id: c85566115777495aa40d6814d0bb7102)
